In [24]:
import time
start_time = time.time()
import cv2
import numpy as np
import math
from PIL import Image
%matplotlib auto
image = cv2.imread('heart.jpg',0)
cv2.imshow('original', image)

Using matplotlib backend: Qt5Agg


In [25]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [26]:
import blowfish
cipher = blowfish.Cipher(b'cipherlendontmatter')

In [27]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher.encrypt_ecb(data))

#assert data == data_decrypted

In [28]:
from SystemParams import *

#KGC generates params
kgc_1 = KGC(256)
params = Params(kgc_1.p, kgc_1.q, kgc_1.g, kgc_1.p_pub)  # params generated by KGC
Alice = User( 'Alice', params )
Bob = User( 'Bob', params )
kgc_1.partialkey_compute(Alice)
kgc_1.partialkey_compute(Bob)

In [29]:
m = int.from_bytes(data_encrypted, "big")
a = m
# break into list if it is too long
m_list = []
while m > 0:
    m_list.append(m % 10000000000)
    m = m // 10000000000

#signcrypt every item in the list
Signcryption_text = []
for i in range(len(m_list)):
    Signcryption_text.append(Alice.signcrypt( Bob, params, m_list[i] ))

In [30]:
#unsigncrypt every item in the list
m_un = []
for i in range(len(Signcryption_text)):
    m_un.append(Bob.unsigncrypt( Alice, params, Signcryption_text[i] ))

#merge list back into one number
m_un.reverse()
m_un_merge = 0
for i in range(len(m_un)):
    m_un_merge = m_un_merge * 10000000000 + m_un[i]

m_un_merge2 = int(m_un_merge)

#convert number back to bytes
m_un_merge_bytes = m_un_merge2.to_bytes(len(data_encrypted), "big")

In [31]:
data_decrypted = b"".join(cipher.decrypt_ecb(m_un_merge_bytes))

In [32]:
img_str_np = np.frombuffer(data_decrypted, dtype=np.uint8)
img = cv2.imdecode(img_str_np, 0)

In [33]:
cv2.imshow('decrypted', img)

In [34]:
#img_str2 = cv2.imencode('.jpg', img)[1].tobytes()
#img_str2 = img_str2[0:len(img_str)]
#img_str_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [35]:
def nearest_square(limit):
    next = math.floor(math.sqrt(limit)) + 1
    return next * next

In [36]:
img_str_np_enc = np.frombuffer(data_encrypted, dtype=np.uint8)
#add elements to the end of the array to make it a square
q = nearest_square(len(img_str_np_enc))

while len(img_str_np_enc) != q:
    img_str_np_enc = np.append(img_str_np_enc, 0)
    

img_str_np_enc1 = img_str_np_enc.reshape(int(math.sqrt(len(img_str_np_enc))), int(math.sqrt(len(img_str_np_enc))))

In [37]:
enc_img = Image.fromarray(img_str_np_enc1)
enc_img.show()


In [38]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(255 / math.sqrt(mse))

d = psnr(image, img)
print(d)

52.7287009949582


In [39]:
cv2.PSNR(image, img)

52.7287009949582

In [40]:
MSE = np.square(np.subtract(img_str_np,og_img_np)).mean()
MSE

0.0

In [41]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 0.3469047619047619


In [42]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99997572],
       [0.99997572, 1.        ]])

In [43]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [44]:
image = image.ravel()
img = img.ravel()
for i in range(len(image)) :
    if image[i] == 0 :
        image[i] = 1
    if img[i] == 0 :
        img[i] = 1

out = np.divide(img, image)
a = np.sum(out)
for i in range(len(out)) :
    out[i] = out[i] / a

entropy(out)

15.6162273779474

In [45]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.259416341781616 seconds ---


In [46]:
entropy(img.ravel())

36692243.94608375